# **LP2_Run1**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are in charge of the supply purchasing of a company that produces two kinds of drugs.
The drugs contain a specific active agent, which is extracted from two different kinds of raw materials that should be purchased on the market.

The goal is to maximize the total profit obtained from producing the drugs, which means minimizing purchasing costs for buying the raw materials and minimizing operational costs for producing the drugs while maximizing sales.

You are given a budget for purchasing raw materials and operating the production process of the drugs which cannot be exceeded.
Additionally, you need to keep the capacity constraints for the production of the drugs in mind as there is only a limited amount of manpower as well as equipment hours available.
Your company can also only store a limited amount of raw materials.
Most importantly, the amount of active ingredient extracted from the raw materials you purchase needs to satisfy the required amount of active ingredient in the drugs your company produces.
Importantly, your experience with the production of the drugs has shown the the amount of active ingredient your process extracts from the two raw materials can vary by up to 0.5% and 2%, respectively.
During purchasing, you need to make sure that no matter how much the amount of active ingredient varies, the required amount for the drug production is always met. """

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [6]:
print(response1.choices[0].message.content)

1. X1: Quantity of first kind of raw material to be purchased.
2. X2: Quantity of second kind of raw material to be purchased.
3. Y1: Quantity of first drug produced.
4. Y2: Quantity of second drug produced.
5. Z1: Quantity of active ingredient extracted from the first raw material.
6. Z2: Quantity of active ingredient extracted from the second raw material.
7. C: Total cost for purchasing raw materials and operation costs.
8. P: Total profit from selling the drugs.
9. B: Budget for the purchasing of raw materials and operation costs.
10. A: Total amount of the active ingredient required for drug production.
11. M: Maximum manpower capacity for drug production.
12. E: Maximum equipment hour capacity for drug production.
13. S: Storage capacity for raw materials.
14. R1: Range of variation in the amount of active ingredient from first raw material (0.5%)
15. R2: Range of variation in the amount of active ingredient from second raw material (2%).


In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [8]:
print(response2.choices[0].message.content)

Max P = Sales price of drug1*Y1 + Sales price of drug2*Y2 - C

Where:

C = Price of raw material1*X1 + Price of raw material2*X2 + Op_cost(Y1,Y2)

Subject to given constraints. (Constraints are not included in the objective function).


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [12]:
print(response3.choices[0].message.content)

1. X1, X2 >= 0 (Cannot purchase negative quantity of raw materials)
2. Y1, Y2 >= 0 (Cannot produce negative quantity of drugs)
3. C <= B (Cost of purchasing raw materials and operation should not exceed the budget)
4. Z1 = X1 - X1*R1 and Z2 = X2 - X2*R2 (Quantity of active ingredient extracted can vary)
5. Z1 + Z2 >= A (Quantity of active agents extracted must satisfy the required amount for drug production)
6. Manpower(Y1, Y2) <= M (Manpower required for production should not exceed the maximum manpower capacity)
7. Equipment_hours(Y1, Y2) <= E (Equipment hours required for production should not exceed maximum equipment hours capacity)
8. X1 + X2 <= S (Storage constraint, cannot purchase more raw materials if storage capacity is full)
9. Ingredients(Y1, Y2) <= Z1 + Z2 (The active ingredients used to produce the drugs should not exceed the actual amount extracted from the raw materials).


### **Generate Pyomo Code**

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [13]:
Markdown(response4.choices[0].message.content)

Here is a potential solution using pyomo. Please note that I use sample data in defining parameters including budget, raw material prices, operating cost per drug, and sales prices. The active ingredient proportions per raw material and the active ingredient requirement per drug are also sample data.

```python
from pyomo.environ import *

model = ConcreteModel()

# Decision Variables
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)
model.Y1 = Var(within=NonNegativeReals)
model.Y2 = Var(within=NonNegativeReals)

# Parameters (Sample Data used)
model.B = Param(initialize=10000) # Budget
model.raw_material_price1 = Param(initialize=20) # Price per unit of raw material 1
model.raw_material_price2 = Param(initialize=25) # Price per unit of raw material 2
model.Op_cost1= Param(initialize=2) # Operating cost per unit of drug 1
model.Op_cost2 = Param(initialize=3) # Operating cost per unit of drug 2
model.price_of_drug1 = Param(initialize=50) # Sales price per unit of drug 1
model.price_of_drug2 = Param(initialize=75) # Sales price per unit of drug 2
model.R1 = Param(initialize=0.005) # Range of variation in active ingredient from raw material 1
model.R2 = Param(initialize=0.02) # Range of variation in active ingredient from raw material 2
model.A1 = Param(initialize=0.2) # Active ingredient requirement per unit of drug 1
model.A2 = Param(initialize=0.15) # Active ingredient requirement per unit of drug 2
model.M = Param(initialize=100) # Maximum manpower
model.E = Param(initialize=200) # Maximum equipment hours
model.S = Param(initialize=500) # Storage capacity

# Objective Function
model.Profit = Objective(expr=model.price_of_drug1*model.Y1 + model.price_of_drug2*model.Y2 - model.raw_material_price1*model.X1 - model.raw_material_price2*model.X2 - model.Op_cost1*model.Y1 - model.Op_cost2*model.Y2, 
                      sense=maximize)

# Constraint
model.C1 = Constraint(expr=model.raw_material_price1*model.X1 + model.raw_material_price2*model.X2 + model.Op_cost1*model.Y1 + model.Op_cost2*model.Y2 <= model.B)
model.C2 = Constraint(expr=model.X1 - model.X1*model.R1 + model.X2 - model.X2*model.R2 >= model.Y1*model.A1 + model.Y2*model.A2)
model.C3 = Constraint(expr=model.X1 + model.X2 <= model.S)

# Call solver
SolverFactory('cbc').solve(model)
return model.Profit()
```

### **Run the code Generated by GPT4**

In [19]:
from pyomo.environ import *

model = ConcreteModel()

# Decision Variables
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)
model.Y1 = Var(within=NonNegativeReals)
model.Y2 = Var(within=NonNegativeReals)

# Parameters (Sample Data used)
model.B = Param(initialize=10000) # Budget
model.raw_material_price1 = Param(initialize=20) # Price per unit of raw material 1
model.raw_material_price2 = Param(initialize=25) # Price per unit of raw material 2
model.Op_cost1= Param(initialize=2) # Operating cost per unit of drug 1
model.Op_cost2 = Param(initialize=3) # Operating cost per unit of drug 2
model.price_of_drug1 = Param(initialize=50) # Sales price per unit of drug 1
model.price_of_drug2 = Param(initialize=75) # Sales price per unit of drug 2
model.R1 = Param(initialize=0.005) # Range of variation in active ingredient from raw material 1
model.R2 = Param(initialize=0.02) # Range of variation in active ingredient from raw material 2
model.A1 = Param(initialize=0.2) # Active ingredient requirement per unit of drug 1
model.A2 = Param(initialize=0.15) # Active ingredient requirement per unit of drug 2
model.M = Param(initialize=100) # Maximum manpower
model.E = Param(initialize=200) # Maximum equipment hours
model.S = Param(initialize=500) # Storage capacity

# Objective Function
model.Profit = Objective(expr=model.price_of_drug1*model.Y1 + model.price_of_drug2*model.Y2 - model.raw_material_price1*model.X1 - model.raw_material_price2*model.X2 - model.Op_cost1*model.Y1 - model.Op_cost2*model.Y2,
                      sense=maximize)

# Constraint
model.C1 = Constraint(expr=model.raw_material_price1*model.X1 + model.raw_material_price2*model.X2 + model.Op_cost1*model.Y1 + model.Op_cost2*model.Y2 <= model.B)
model.C2 = Constraint(expr=model.X1 - model.X1*model.R1 + model.X2 - model.X2*model.R2 >= model.Y1*model.A1 + model.Y2*model.A2)
model.C3 = Constraint(expr=model.X1 + model.X2 <= model.S)

# Call solver
SolverFactory('glpk').solve(model)
return model.Profit()

SyntaxError: 'return' outside function (<ipython-input-19-ef96d9b7073b>, line 38)

In [22]:
from pyomo.environ import *

model = ConcreteModel()

# Decision Variables
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)
model.Y1 = Var(within=NonNegativeReals)
model.Y2 = Var(within=NonNegativeReals)

# data inputted by human
model.B = Param(initialize=10000) # Budget
model.raw_material_price1 = Param(initialize=100) # Price per unit of raw material 1
model.raw_material_price2 = Param(initialize=199.90) # Price per unit of raw material 2
model.Op_cost1= Param(initialize=700) # Operating cost per unit of drug 1
model.Op_cost2 = Param(initialize=800) # Operating cost per unit of drug 2
model.price_of_drug1 = Param(initialize=6200) # Sales price per unit of drug 1
model.price_of_drug2 = Param(initialize=6900) # Sales price per unit of drug 2
model.R1 = Param(initialize=0.005) # Range of variation in active ingredient from raw material 1
model.R2 = Param(initialize=0.02) # Range of variation in active ingredient from raw material 2
model.A1 = Param(initialize=0.5) # Active ingredient requirement per unit of drug 1
model.A2 = Param(initialize=0.6) # Active ingredient requirement per unit of drug 2
model.M = Param(initialize=2000) # Maximum manpower
model.E = Param(initialize=800) # Maximum equipment hours
model.S = Param(initialize=1000) # Storage capacity
#end
# Objective Function
model.Profit = Objective(expr=model.price_of_drug1*model.Y1 + model.price_of_drug2*model.Y2 - model.raw_material_price1*model.X1 - model.raw_material_price2*model.X2 - model.Op_cost1*model.Y1 - model.Op_cost2*model.Y2,
                      sense=maximize)

# Constraint
model.C1 = Constraint(expr=model.raw_material_price1*model.X1 + model.raw_material_price2*model.X2 + model.Op_cost1*model.Y1 + model.Op_cost2*model.Y2 <= model.B)
model.C2 = Constraint(expr=model.X1 - model.X1*model.R1 + model.X2 - model.X2*model.R2 >= model.Y1*model.A1 + model.Y2*model.A2)
model.C3 = Constraint(expr=model.X1 + model.X2 <= model.S)

# Call solver
SolverFactory('glpk').solve(model)
print(model.Profit())
print(model.X1(), model.X2(), model.Y1(), model.Y2())

72638.98191560605
6.69792364367046 0.0 13.3288680509042 0.0


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**